In [5]:
include("cumulant.jl")

decay (generic function with 1 method)

In [6]:
bath=overdampedBath(1.0,0.05,5.0);

In [7]:
spectral_density(bath,1)

0.019230769230769232

In [8]:
x=LinRange(0,10,1000);

In [10]:
@time decay.(1,2,bath,x) 

  0.431373 seconds (18.89 k allocations: 10.937 MiB, 3.43% gc time)


1000-element Vector{ComplexF64}:
                    0.0 - 0.0im
 0.00020229067320131277 - 1.0124742860381548e-6im
  0.0006728950461714012 - 6.735911129470973e-6im
  0.0013362473890907074 - 2.006528254413994e-5im
    0.00215413533150333 - 4.313159501487642e-5im
   0.003098836562768099 - 7.75646548780272e-5im
   0.004152653883018949 - 0.0001247418205696109im
   0.005300962229701718 - 0.0001857954220831793im
   0.006530771685797982 - 0.0002616321913998972im
   0.007832431007283265 - 0.00035305102662004683im
                        ⋮
   -0.10679335746373063 - 0.422515402730825im
   -0.10880740145682652 - 0.4215908536660643im
   -0.11081174645379478 - 0.4206448867118278im
   -0.11280693419885117 - 0.4196804372953477im
   -0.11479198754488046 - 0.41869467700752117im
    -0.1167671632755573 - 0.41768941490094413im
   -0.11873331825373387 - 0.41666843506113965im
   -0.12068714447697373 - 0.41562090206748825im
   -0.12263068667625694 - 0.41455487653540984im

In [9]:
3.79/0.420352 #blas #only 8x with respect to simple cython and quad, worth it?

9.01625304506699

In [11]:
3.01/0.4313 #mkl

6.978900996985856